In [ ]:
import pandas as pd
import re

In [5]:
proofnet = pd.read_csv('/Users/aditya/neural-proof-assistant/backend/data/proofnet/proofnet.csv')
proofnet['combined_text'] = proofnet['nl_proof'] + " " + proofnet['first_step']
proofnet

nl_statement  ...                                      combined_text
0    If $r$ is rational $(r \neq 0)$ and $x$ is irr...  ...  \begin{proof}\n\n    If $r$ and $r+x$ were bot...
1    Prove that there is no rational number whose s...  ...  \begin{proof}\n\n    Suppose $m^2=12 n^2$, whe...
2    Let $A$ be a nonempty set of real numbers whic...  ...  \begin{proof}\n\n    We need to prove that $-\...
3    If $z$ is a complex number, prove that there e...  ...  \begin{proof}\n\n    If $z=0$, we take $r=0, w...
4    If $x, y$ are complex, prove that $||x|-|y|| \...  ...  \begin{proof}\n\n    Since $x=x-y+y$, the tria...
..                                                 ...  ...                                                ...
179  Prove that $(x, y)$ is not a principal ideal i...  ...  \begin{proof}\n\n    Suppose, to the contrary,...
180  Prove that if $f(x)$ and $g(x)$ are polynomial...  ...  \begin{proof}\n\n    Let $f(x), g(x) \in \math...
181  Prove that $x^6+30x^5-15x^3 + 6x-120$ is irred...  ...  \begin{proof}\n\n    $$\n\nx^6+30 x^5-15 x^3+6...
182  Prove that $\frac{(x+2)^p-2^p}{x}$, where $p$ ...  ...  \begin{proof}\n\n$\frac{(x+2)^p-2^p}{x} \quad ...
183  Prove that $x^2+y^2-1$ is irreducible in $\mat...  ...  \begin{proof}\n\n$$\n\np(x)=x^2+y^2-1 \in Q[y]...

[184 rows x 5 columns]

In [9]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text) # removes punctuation
    text = re.sub(r'\s+', ' ', text) # remove extra whitespace
    return text.strip()

def remove_duplicate_halves(text):
    words = text.split()
    half = len(words) // 2
    if words[:half] == words[half:]:
        return ' '.join(words[:half])
    return text

In [14]:
proofnet['cleaned_text'] = proofnet['combined_text'].apply(clean_text).apply(remove_duplicate_halves)
proofnet['cleaned_text']

0      beginproof if r and rx were both rational then...
1      beginproof suppose m212 n2 where m and n have ...
2      beginproof we need to prove that sup a is the ...
3      beginproof if z0 we take r0 w1 in this case w ...
4      beginproof since xxyy the triangle inequality ...
                             ...                        
179    beginproof suppose to the contrary that x yp f...
180    beginproof let fx gx in mathbbqx be such that ...
181    beginproof x630 x515 x36 x120 the coefficients...
182    beginproof fracx2p2px quad quad p is on add pp...
183    beginproof pxx2y21 in qyx cong qy x we have th...
Name: cleaned_text, Length: 184, dtype: object

In [16]:
from sklearn.model_selection import train_test_split

X = proofnet['cleaned_text']
y = proofnet['tactic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
proofnet['clean_nl'] = proofnet['nl_proof'].apply(clean_text)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10) # limit to only 10 for now
X_tf_idf = vectorizer.fit_transform(proofnet['clean_nl'])
dense_array = X_tf_idf.toarray()
feature_names = vectorizer.get_feature_names_out()
print(X_tf_idf[2]) # Sparse vector
print(dense_array[2])


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10 stored elements and shape (1, 10)>
  Coords	Values
  (0, 0)	0.21836878903855272
  (0, 8)	0.085403212127263
  (0, 6)	0.23226833062359684
  (0, 9)	0.16980410014829447
  (0, 5)	0.09707475914085646
  (0, 3)	0.33691588109850623
  (0, 4)	0.597822484890841
  (0, 7)	0.15224392266544856
  (0, 1)	0.5011929939823893
  (0, 2)	0.32793167131133405
[0.21836879 0.50119299 0.32793167 0.33691588 0.59782248 0.09707476
 0.23226833 0.15224392 0.08540321 0.1698041 ]


In [35]:
def print_vector_with_words(vector, feature_names, threshold=0.0, sort=True):
    word_score_pairs = [
        (word, val) for word, val in zip(feature_names, vector) if val > threshold
    ]
    if sort:
        word_score_pairs = sorted(word_score_pairs, key=lambda x: -x[1])
    for word, score in word_score_pairs:
        print(f"{word:<15} : {score:.4f}")
print_vector_with_words(dense_array[2], feature_names)

of              : 0.5978
for             : 0.5012
is              : 0.3369
in              : 0.3279
that            : 0.2323
and             : 0.2184
we              : 0.1698
the             : 0.1522
so              : 0.0971
then            : 0.0854
